In [278]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from ortools.linear_solver import pywraplp
import matplotlib.pyplot as plt
from tqdm import tqdm

In [301]:
mus_data={'sector 1':0.15,'sector 2':0.10,'sector 3':0.05}

mus = pd.Series(mus_data)

var_data={'sector 1':0.04,'sector 2':0.04,'sector 3':0.04}
var = pd.Series(var_data)

cov_data={'sector 1':[.04,0.02,0.0],'sector 2':[.02,.04,0.0],'sector 3':[0.0,0,.04]}
cov=pd.DataFrame(cov_data,index=['sector 1','sector 2','sector 3'])

In [302]:
mus,var,cov

(sector 1    0.15
 sector 2    0.10
 sector 3    0.05
 dtype: float64,
 sector 1    0.04
 sector 2    0.04
 sector 3    0.04
 dtype: float64,
           sector 1  sector 2  sector 3
 sector 1      0.04      0.02      0.00
 sector 2      0.02      0.04      0.00
 sector 3      0.00      0.00      0.04)

# Find portfolio weights that maximise Sharpe 
## Weights must add to 1 and sector 2 must be at least 30% weight of portfolio

In [303]:
n_assets = 3

mean_variance_pairs = []
mean=[]
weights_list=[]
tickers_list=[]
portfolio_weights=[]
for i in tqdm(range(5000)):
    next_i = False
    while True:
        assets = ['sector 1','sector 2','sector 3']
        y=np.array([0,0.3,0])
        x=np.random.dirichlet(np.ones(3))*.5
        weights=(y+x)
        weights = weights/sum(weights)
        
        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * mus.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
        
        for R,V in mean_variance_pairs:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break
        if next_i:
            break
        weights.tolist()
        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
        portfolio_weights.append([portfolio_E_Return,portfolio_E_Variance,weights])
        weights_list.append(weights)
        tickers_list.append(assets)
        break

100%|█████████████████████████████████████| 5000/5000 [00:01<00:00, 4283.09it/s]


In [304]:
newlist=mean_variance_pairs

In [305]:
risk_free_rate=0 #risk free rate
sharpe=[]
for i in range(0, len(newlist)):
    sharpe.append((newlist[i][0]-risk_free_rate)/(newlist[i][1]**.5))
max(sharpe)
np.argmax(sharpe)
mean_variance_pairs[np.argmax(sharpe)]
# portfolio_weights[164]
newlist[np.argmax(sharpe)]

[0.11705811422230857, 0.023029611758748385]

In [306]:
newlist = np.array(newlist)

risk_free_rate=0

fig = go.Figure()
fig.add_trace(go.Scatter(x=newlist[:,1]**0.5, y=newlist[:,0], 
                      marker=dict(color=(newlist[:,0]-risk_free_rate)/(newlist[:,1]**.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe Ratio")
                                 ), 
                      mode='markers',
                      text=[str(np.array(tickers_list[i])) + "" + str(np.array(weights_list[i]).round(2)) for i in range(len(tickers_list))]))


fig.add_trace(go.Scatter(x=[mean_variance_pairs[np.argmax(sharpe)][1]**.5],
                 y=[mean_variance_pairs[np.argmax(sharpe)][0]],
             mode="markers",marker_color="lightskyblue",marker_size=15))




fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk'),
                  yaxis=dict(title='Annualised Return'),
                  title='Efficient Frontier',
                  width=850,
                  height=500)
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

## Maximum Sharpe portfolio has weights:
### Sector 1: 0.48
### Sector 2: 0.38
### Sector 3: 0.14
### With a Sharpe ratio of 0.7713

In [312]:
weights_list[np.argmax(sharpe)]

array([0.48254573, 0.37607082, 0.14138345])

In [313]:
np.max(sharpe)

0.7713619555989948